In [ ]:
%%configure -f
{
    "kind": "pyspark",
    "name": "edwardchen",

    // Driver configuration
    "driverMemory": "2G",
    "driverCores": 1,

    // Executor configuration
    "executorCores": 4,
    "executorMemory": "8G",
    "numExecutors": 4,

    // Additional runtime parameters
    "heartbeatTimeoutInSecond": 120,

    // Load dependencies and environment archive
    "pyFiles": ["hdfs://path/data/*"],

    // YARN queue
    "queue": "default",
    "archives": ["hdfs://path/zip/base.zip#python_env"],

    // Spark configuration
    "conf": {
        "spark.default.parallelism": "16",
        "spark.sql.shuffle.partitions": "200",
        "spark.sql.auto.repartition": "true",
        "spark.yarn.appMasterEnv.PYSPARK_PYTHON": "python_env/bin/python"
     }
}

In [ ]:
from pyspark.sql import functions as F
from pyspark.sql.window import Window
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()
# spark
# spark.version

# Time distribution of first purchase after register


In [ ]:
users = spark.read.parquet('/path/to/data')
users = users.select('REG_DATE','ID', 'USER_STATUS','USER')\
             .filter(
                    (F.col('REG_DATE').between('20230401', '20230531')) &
                    (F.col('USER_STATUS').isin([0,6])) &
                    (~F.col('USER').rlike('@random')))\
             .drop('USER_STATUS','USER')

orders = spark.read.parquet('/path/to/data')
orders = orders.select('P_ID','ORDER_DATE', 'G_CLASS', 'G_NAME', 'G_NO', 'TOTAL_AMOUNT')\
              .withColumnRenamed('P_ID', 'ID')

# users.show(5, truncate=False)
# orders.show(5, truncate=False)

In [ ]:
joined_df = users.join(orders, on='ID', how='left')
joined_df = joined_df.filter(F.col("REG_DATE").isNotNull())\
                     .dropDuplicates()\
                     .withColumn("DATE", F.to_date("ORDER_DATE", "yyyyMMdd"))\
                     .withColumn("REG_DATE", F.to_date("REG_DATE", "yyyyMMdd"))\
                     .withColumn('TIME_DIF', F.datediff("DATE", "REG_DATE"))\
                     .cache()

# joined_df.show(5, truncate=False)
# joined_df.printSchema()

In [ ]:
window_spec = Window.partitionBy("ID").orderBy("DATE")
joined_df = joined_df.withColumn("row_num", F.row_number().over(window_spec))
count_df = joined_df.filter(F.col("row_num") == 1).drop("row_num")

print("Number of rows in df:", count_df.count())

In [ ]:
%spark -o count_df -n 30000

In [ ]:
%local

%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt

# Percent NA
total_count = len(count_df)
count_NaN = count_df["TIME_DIF"].isna().sum()
percentage_NaN = (count_NaN / total_count) * 100
print(percentage_NaN)

## By date
for day in [0, 7, 14, 30, 60, 90, 180]:
    percentage = (count_df["TIME_DIF"] <= day).sum() / total_count * 100
    print(percentage)

value_counts = count_df["TIME_DIF"].value_counts().sort_index()

# Plot using Matplotlib
plt.figure(figsize=(10, 5))
plt.bar(value_counts.index, value_counts.values)
plt.xlabel('Days (TIME_DIF)')
plt.ylabel('Number of Users')
plt.title('Time to First Purchase Distribution')
plt.xlim(0, 30)

plt.savefig("graph.png", dpi=300, bbox_inches='tight')
plt.show()

# Save count_df
count_df.to_csv('output.csv', index=False)

# Average Order and Amount


In [ ]:
trans = spark.read.parquet('/path/to/data')
trans = trans.select('TRANSACTION_NO', 'P_ID', F.col('TOTAL_AMOUNT').cast('double'), 'INSERT_TIME', 'TRANS_CANCELFLAG')\
             .withColumn("DATE", F.to_date("INSERT_TIME", "yyyyMMdd"))\
             .filter(F.col("DATE").between("2023-04-01", "2023-05-31"))\
             .filter(F.col("TRANS_CANCELFLAG") == F.lit("N"))\
             .withColumnRenamed("P_ID", "ID") \
             .filter(F.col("TOTAL_AMOUNT") <= 200000) \
             .drop("INSERT_TIME") \
             .cache()

#trans.show(5, truncate=False)
#trans.agg(sum("TOTAL_AMOUNT").alias("total_sales")).show()

In [ ]:
# Daily purchase amount and order
df = trans.join(joined_df, on=["ID", "DATE"], how='inner')

#Check duplicate
duplicate_count = df.groupBy("TRANSACTION_NO").count().filter(F.col("count") > 1)
if duplicate_count.count() > 0:
    print("Duplicate values found in the 'TRANSACTION_NO' column:")
    duplicate_count.show()
else:
    print("No duplicate values found in the 'TRANSACTION_NO' column.")

df = df.dropDuplicates(["TRANSACTION_NO"])

In [ ]:
sum_df = df.groupBy("ID", "DATE").agg(F.sum("TOTAL_AMOUNT").alias("TOTAL_AMOUNT_DAY"),
                                      F.count("TRANSACTION_NO").alias("NUM_TRANS"),
                                      F.first("TIME_DIF", True).alias("TIME_DIF"))\
                                      .withColumn("TIME_DIF_FIRST", F.first("TIME_DIF").over(window_spec))\
                                      .cache()

#sum_df.show()

# Repurchase and Customer Lifetime Value

In [ ]:
value_df = sum_df.groupBy("TIME_DIF_FIRST").agg(F.sum("TOTAL_AMOUNT_DAY").alias("Sales_By_TIME_DIF"),
                                                F.countDistinct("ID").alias("PEOPLE_COUNT"))\
                                                .orderBy(F.asc("TIME_DIF_FIRST"))

distribution = value_df.describe("PEOPLE_COUNT")
distribution.show()

# Calculate quantile
quantiles_result = value_df.approxQuantile("PEOPLE_COUNT", [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9], 0.01)
for i, q_val in enumerate(range(10, 100, 10)):
    print(f"Quantile {q_val}%: {quantiles_result[i]}")

# Quantile 10%: 43.0
# Quantile 20%: 234.0
# Quantile 30%: 801.0
# Quantile 40%: 2301.0
# Quantile 50%: 4940.0
# Quantile 60%: 8083.0
# Quantile 70%: 12807.0
# Quantile 80%: 18519.0
# Quantile 90%: 26754.0

 # Total Sales of First-Time Buyer Products -(start here)

In [ ]:
# Include First-Time Purchasers Within 10 Days of Registration
# .orderBy(F.col("TOTAL_AMOUNT").desc()) =  .orderBy(F.desc("TOTAL_AMOUNT")
goodno_df = df.withColumn("TIME_DIF_FIRST", F.first("TIME_DIF", True).over(window_spec))\
              .filter(F.col("TIME_DIF").isNotNull())\
              .filter(F.col("TIME_DIF") == F.col("TIME_DIF_FIRST"))\
              .filter(F.col('TIME_DIF_FIRST') <= 10)\
              .withColumn("G_ClASS4", F.substring(F.col("G_CLASS"), 1, 4))\
              .groupBy("G_NO").agg(F.sum("TOTAL_AMOUNT").alias("TOTAL_AMOUNT"),
                                   F.first("G_NAME").alias("Name"),
                                   F.first("G_ClASS4").alias("G_ClASS4"))\
              .orderBy(F.desc("TOTAL_AMOUNT"))

#goodno_df.limit(100).show(10)

In [ ]:
%spark -o goodno_df  -n 100
%spark -o value_df   -n 1500

In [ ]:
%local
import pandas as pd

# Set display options to show all rows and columns
pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)
pd.set_option("display.width", None)

display(goodno_df)
display(value_df)

##Log Data

In [ ]:
pv_example = spark.read.parquet("/user/pv_202304*")
pv_example5 = spark.read.parquet("/user/pv_202305*")
pv_combined = pv_example.unionByName(pv_example5, allowMissingColumns=True)
print (pv_combined.count(),pv_combined.columns)

pv = pv_combined.filter((F.col("NAME").like("%Diet%") & F.col("NAME").like("%Soda%")) | (F.col("NAME").like("%Zero%") & F.col("NAME").like("%Soda%")))\
                .filter(F.col('TYPE').isin('Search', 'Online', 'Amazon'))\
                .withColumn("G_NO", F.get_json_object(F.col("col_name"), "$.key"))\
                .fillna(0, subset=["AMOUNT"])\
                .dropDuplicates(["ID", "DATE"])

names = pv.select("NAME").distinct().limit(100).collect()
for row in names:
    print(row["NAME"])

In [ ]:
#count
items = ["20240614", "20240615", "20240616"]

filter_condition = F.col("G_NO").contains(items[0])
for item in items[1:]:
     filter_condition |= F.col("G_NO").contains(item)

pv = pv.filter(filter_condition)

for item in items:
    count_result = pv.filter(F.col("G_NO").contains(item)) \
                     .agg(F.count("*").alias("count")) \
                     .collect()[0]["count"]
    print(f"{item}: {count_result}")

In [ ]:
result = (
    pv.join(orders, (orders["DATE"] == pv["DATE"]) & (orders["ID"] == pv["CTRL_ROWID"]), how='left')\
      .groupBy("ID")\
      .agg(
          F.first("SELLER_NAME").alias("SELLER_NAME"),
          F.sum(orders["TOTAL_AMOUNT"]).alias("GMV"),
          F.avg(F.when(F.col("MONTH").between(4, 5), orders["TOTAL_AMOUNT"])).alias("AVG_GMV"),
          F.sum(F.when(F.col("G_NO") == "(null)", 1).otherwise(0)).alias("on_stock"),
          F.min(pv["DATE"]).alias("EARLIEST_DATE"),
          F.count('*').alias('COUNT'),
          F.avg(orders["TOTAL_AMOUNT"]).alias("AVERAGE_PRICE").alias("AVERAGE_PRICE")))\
       .withColumn("GMV_bin",
             F.when(F.col("GMV") < 1000, "Low")
            .when(F.col("GMV").between(1000, 5000), "Medium")
            .otherwise("High"))\
      .orderBy("ID")

total_rows = result.count()
result = result.withColumn("percentage", (F.col("COUNT") / total_rows) * 100)

In [ ]:
gmv_month = result.withColumn("EARLIEST_MONTH", F.month(F.col("EARLIEST_DATE")))\
                  .groupBy("EARLIEST_MONTH")\
                  .pivot("GMV_bin", ["Low", "Medium", "High"])\
                  .agg(F.count("*"))\
                  .fillna(0)\
                  .orderBy("EARLIEST_MONTH")

#gmv_month.show(30, truncate=False)
# +--------------+-----+------+-----+
# |EARLIEST_MONTH|Low  |Medium|High |
# +--------------+-----+------+-----+
# |1             |10   |5     |2    |
# |2             |8    |6     |1    |
# |3             |15   |8     |5    |
# |4             |12   |10    |4    |
# +--------------+-----+------+-----+

## Document

In [ ]:
# Define a window partitioned by id, ordered by date
# window_spec = Window.partitionBy("ID").orderBy("DATE")

# Example DataFrame
# df: ID | DATE | CATEGORY | VALUE

# df_new = df.withColumn("SUM",          F.sum("VALUE").over(window_spec))\                                       # Cumulative sum of flag per user
#            .withColumn("MAX_VALUE",    F.max("VALUE").over(window_spec))\                                       # Max value per user
#            .withColumn("CATEGORY",     F.lead("CATEGORY").over(window_spec))\                                   # Next category in order for each user
#            .withColumn("MIN_VALUE",    F.min("VALUE").over(window_spec))\                                       # Minimum value per user
#            .withColumn("SECOND_ROW",   F.when(F.row_number().over(window_spec) == 1, 1).otherwise(0))\          # Mark first row
#            .withColumn("FIRST_DATE_C", F.first(F.when("VALUE" <= 1000, F.col("DATE")), True).over(window_spec)) # First non-null date where VALUE <= 1000
#            .withColumn("LAST_VALUE",   F.last("VALUE").over(window_spec))\                                      # Return last non-null value in window partition

# F.lpad(string, 2, "0")                                      # Pad a string on the left to length 2 with character "0"
# F.month(date)                                               # Extract the month from a DateType or Timestamp
# F.dayofmonth(date)                                          # Extract the day of the month from a DateType or Timestamp
# F.year(date)                                                # Extract year from a DateType or Timestamp
# F.concat(string_1, string_2, string_3)                      # Combine multiple StringType columns into one
# F.unix_timestamp(string, "format")                          # Convert a string with specified format to Unix timestamp
# F.from_unixtime(timestamp, "format")                        # Convert a Unix timestamp to formatted date string
# F.date_format(date, "format")                               # Format a DateType or TimestampType column into a string with specified pattern
# (F.col("boolean_1") | F.col("boolean_2"))                   # Logical OR between two BooleanType columns
# F.to_date("20230401", "yyyyMMdd")                           # Convert string to DateType; format "yyyyMMdd" for strings like "20230401", "yyyy-MM-dd" for "2023-04-01"
# F.to_timestamp("2023-04-01 14:30:00","yyyy-MM-dd HH:mm:ss") # Convert string to TimestampType with specified format
# F.substring(F.col("text"), -10, 10)                         # Extract last 10 characters (negative index counts from end)
# F.col("VALUE").isNull()                                     # Check if column value is NULL
# F.isnan(F.col("VALUE"))                                     # Check if numeric column value is NaN (Not a Number)